In [1]:
#Import required libraries
from random import seed
from random import randrange
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
#Read txt file
data= pd.read_csv("C:/Users/Saani Ausaf/Documents/Fall_2017/DM/Project3/project3_dataset2.txt", sep="\t", header=None)
print(data.head())

     0     1     2      3        4   5      6      7   8  9
0  132  6.20  6.47  36.21  Present  62  30.77  14.14  45  0
1  123  0.05  4.61  13.69   Absent  51  23.23   2.78  16  0
2  128  0.50  3.70  12.81  Present  66  21.25  22.73  28  0
3  114  9.60  2.51  29.18   Absent  49  25.67  40.63  46  0
4  150  0.30  6.38  33.99  Present  62  24.64   0.00  50  0


In [3]:
# Separate Attribute and Label
labels =data[data.columns[-1]]
data = data.ix[:, :len(data.columns)-2]
print(data.head())

     0     1     2      3        4   5      6      7   8
0  132  6.20  6.47  36.21  Present  62  30.77  14.14  45
1  123  0.05  4.61  13.69   Absent  51  23.23   2.78  16
2  128  0.50  3.70  12.81  Present  66  21.25  22.73  28
3  114  9.60  2.51  29.18   Absent  49  25.67  40.63  46
4  150  0.30  6.38  33.99  Present  62  24.64   0.00  50


In [4]:
# Handeling Categorical Data

#Select Categorical Columns from the dataframe
obj_df = data.select_dtypes(include=['object']).copy()

#One-Hot Encoding for categorical data
for idx in obj_df.columns:
    obj_df= pd.get_dummies(obj_df,columns=[idx],drop_first=False)
print(obj_df.head())

obj_dataset = obj_df.values.tolist()

   4_Absent  4_Present
0         0          1
1         1          0
2         0          1
3         1          0
4         0          1


In [5]:
# Handeling Numerical Data
#Select numerical data from the data
num_df = data.select_dtypes(include=['int64','float64']).copy()
print(num_df.head())

# Normalize Numeric Data
if (len(num_df.columns) !=0):
    num_dataset = preprocessing.normalize(num_df, norm='l2')
    num_dataset
else:
    num_dataset = []
print(len(num_dataset))

     0     1     2      3   5      6      7   8
0  132  6.20  6.47  36.21  62  30.77  14.14  45
1  123  0.05  4.61  13.69  51  23.23   2.78  16
2  128  0.50  3.70  12.81  66  21.25  22.73  28
3  114  9.60  2.51  29.18  49  25.67  40.63  46
4  150  0.30  6.38  33.99  62  24.64   0.00  50
462


In [6]:
# Dataset Format to be used for algorithm
if len(num_dataset)!= 0:
    dataset = np.column_stack((num_dataset, obj_dataset))
    dataset = np.column_stack((dataset, labels))
else:
    dataset = np.column_stack((obj_dataset, labels))
(len(dataset[0]))
label_col = len(dataset[0])

In [7]:
#Set Values
n_folds = 10
max_depth = 3

In [8]:
#Implement 10k-fold cross validation 
def cross_validation(dataset, n_folds):
    splitted_dataset = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            idx = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(idx))
        splitted_dataset.append(fold)
    return splitted_dataset

In [9]:
# Split a dataset based on given attribute value
def split(idx, val, dataset):
    lft, rt = list(), list()
    for row in dataset:
        if row[idx] < val:
            lft.append(row)
        else:
            rt.append(row)
    return lft, rt   

# Calculate the Gini index for splitted dataset
def gini_idx(attr_grps, classes):
    # count all possible samples at split point
    total_instances = float(sum([len(attr) for attr in attr_grps]))
    #sum weighted Gini index for each group
    gini = 0.0
    for attr in attr_grps:
        size = float(len(attr))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in attr].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / total_instances)
    return gini


# Select the best split point for a dataset with different attributes
def get_split(dataset, attr_list):
    class_val = list(set(row[-1] for row in dataset))
    b_idx, b_val, b_gini_score, b_attr_grp = 9999, 9999, 9999, None
    for idx in range(len(dataset[0])-1):
        #Use only the attributes not used as nodes
        if idx not in attr_list:
            for row in dataset:
                attr_grps = split(idx, row[idx], dataset) 
                gini = gini_idx(attr_grps, class_val)
                if gini < b_gini_score:
                    b_idx, b_val, b_gini_score, b_attr_grp = idx, row[idx], gini, attr_grps
    return {'idx':b_idx, 'a_val':b_val, 'grps':b_attr_grp, 'gini':  b_gini_score}

In [10]:
# Create a terminal node value
def terminal(attr_grp):
    results = [row[-1] for row in attr_grp]
    return max(set(results), key=results.count)

# Create child splits or terminal nodes
def node_split(node, depth, attr_list, max_depth):
    lft, rt = node['grps']
    del(node['grps'])
    
    # check for a no split
    if not lft or not rt:
        node['lft'] = node['rt'] = terminal(lft + rt)
        return
    
    # check for max depth
    if depth >= max_depth:
        node['lft'], node['rt'] = terminal(lft), terminal(rt)
        return
    
    # process only left child
    if len(attr_list) == len(lft[0])-1:
        node['lft'] = terminal(lft)
    else:
        node['lft'] = get_split(lft, attr_list)
        attr_list.append(node['lft']['idx'])
        node_split(node['lft'], depth+1,attr_list, max_depth)
        
    # process only right child
    if len(attr_list) == len(rt[0])-1:
        node['rt'] = terminal(rt)
    else:
        #print(len(rt[0]), len(attr_list))
        node['rt'] = get_split(rt,attr_list)
        attr_list.append(node['rt']['idx'])
        node_split(node['rt'], depth+1, attr_list, max_depth)
        
# Build a decision tree
def build_tree(train_dataset, attr_list, max_depth):
    root = get_split(train_dataset, attr_list)
    attr_list.append(root['idx'])
    node_split(root, 1,attr_list, max_depth)
    return root, attr_list

# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('SplitNode: [Atrribute: %d, Value: %.4f]' % ((node['idx']), node['a_val']))
        print_tree(node['lft'], depth+1)
        print_tree(node['rt'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [11]:
# Prediction with a decision tree
def predict(node, row):
    if row[node['idx']] < node['a_val']:
        if isinstance(node['lft'], dict):
            return predict(node['lft'], row)
        else:
            return node['lft']
    else:
        if isinstance(node['rt'], dict):
            return predict(node['rt'], row)
        else:
            return node['rt']

In [12]:

def decision_tree(train_dataset, test_dataset, attr_list, max_depth):#, max_depth, min_size):
    #tree, attr = build_tree(train_dataset, max_depth, min_size, attr_list)
    tree, attr = build_tree(train_dataset, attr_list, max_depth)
    print_tree(tree)
    print("_____________________________________________")
    predictions = list()
    for row in test_dataset:
        predicts = predict(tree, row)
        predictions.append(predicts)
    return(predictions)

In [13]:
#Calculate True Positives, False Positive, true negative and False negative
def positives(actual, predicted):
    tp = 0
    fp = 0
    for i in range(len(actual)):
        if actual[i] == 1:
            if predicted[i] == 1:
                tp += 1
            else:
                fp +=1
    return tp, fp

def negatives(actual, predicted):
    tn = 0
    fn = 0
    for i in range(len(actual)):
        if actual[i] == 0:
            if predicted[i] == 0:
                tn += 1
            else:
                fn += 1
    return tn, fn

In [14]:
#Calculate Precision
def precision_recall_F1_accuracy(actual, predicted):
    tr_pos, fal_pos = positives(actual, predicted)
    tr_neg, fal_neg = negatives(actual, predicted)
    
    if (tr_pos + fal_pos) != 0:
        precision = tr_pos/(tr_pos + fal_pos)
    else:
        print("Zero Division\n")
        precision = 0
    
    if (tr_pos+fal_neg) !=0:
        recall = tr_pos/(tr_pos+fal_neg)
    else:
        print("Zero Division\n")
        recall = 0
    
    if (precision +recall) !=0:
        f1 = 2 * (precision * recall)/(precision +recall)
    else: 
        print("Zero Division\n")
        f1 = 0
        
    if (tr_pos + tr_neg + fal_pos + fal_neg) != 0:
        accuracy = (tr_pos +tr_neg)/(tr_pos + tr_neg + fal_pos + fal_neg)
    else:
        print("Zero Division\n")
    
    return precision, recall, f1, accuracy

In [15]:
# Algorithm evaluation with a k-fold cross validation
def eval_algo(dataset, algo, n_folds, max_depth):#, max_depth, min_size):
    folds = cross_validation(dataset, n_folds)
    #scores = list()
    precision = list()
    recall = list()
    F_1_score = list()
    Accuracy = list()
    for fold in range(len(folds)):
        attr_list=[]
        print("Training Data Fold: " + str(fold+1))
        train_dataset = list(folds)
        train_dataset.pop(fold)
        train_dataset = sum(train_dataset, [])
        test_dataset = list()
        for row in folds[fold]:
            row_copy = list(row)
            test_dataset.append(row_copy)
            row_copy[-1] = None
        predicted = algo(train_dataset, test_dataset, attr_list, max_depth)
        actual = [row[-1] for row in folds[fold]]
        P, R, F_1, Acc = precision_recall_F1_accuracy(actual, predicted)
        precision.append(P)
        recall.append(R)
        F_1_score.append(F_1)
        Accuracy.append(Acc)
    return Accuracy, precision, recall, F_1_score

In [16]:
# Test dataset
seed(1234)
# evaluate algorithm
Accuracy, precision, recall, F_1 = eval_algo(dataset, decision_tree, n_folds, max_depth)#, max_depth, min_size)

Training Data Fold: 1
SplitNode: [Atrribute: 7, Value: 0.2473]
SplitNode: [Atrribute: 4, Value: 0.4899]
SplitNode: [Atrribute: 5, Value: 0.1330]
   [0.0]
   [0.0]
SplitNode: [Atrribute: 0, Value: 0.7878]
   [1.0]
   [1.0]
SplitNode: [Atrribute: 1, Value: 0.0525]
SplitNode: [Atrribute: 8, Value: 1.0000]
   [1.0]
   [0.0]
SplitNode: [Atrribute: 6, Value: 0.1247]
   [1.0]
   [1.0]
_____________________________________________
Training Data Fold: 2
SplitNode: [Atrribute: 7, Value: 0.2473]
SplitNode: [Atrribute: 1, Value: 0.0085]
SplitNode: [Atrribute: 5, Value: 0.1455]
   [0.0]
   [0.0]
SplitNode: [Atrribute: 2, Value: 0.0149]
   [1.0]
   [0.0]
SplitNode: [Atrribute: 8, Value: 1.0000]
SplitNode: [Atrribute: 4, Value: 0.3942]
   [1.0]
   [1.0]
SplitNode: [Atrribute: 6, Value: 0.2005]
   [0.0]
   [0.0]
_____________________________________________
Training Data Fold: 3
SplitNode: [Atrribute: 7, Value: 0.2473]
SplitNode: [Atrribute: 4, Value: 0.4899]
SplitNode: [Atrribute: 1, Value: 0.0027]
 

In [17]:
print('Accuracy \t Precision \t Recall \t F_1')
for i in range(len(Accuracy)):
    print('%.5f\t\t' % Accuracy[i] + ' %.5f\t' % precision[i] + ' %.5f\t' % recall[i] + ' %.5f\t' % + F_1[i])
print('\nMean Accuracy:\t %.4f' % (sum(Accuracy)/float(len(Accuracy))))
print('Mean Precision:\t %.4f' % (sum(precision)/float(len(precision))))
print('Mean Recall:\t %.4f' % (sum(recall)/float(len(recall))))
print('Mean F_1:\t %.4f' % (sum(F_1)/float(len(F_1))))

Accuracy 	 Precision 	 Recall 	 F_1
0.73913		 0.75000	 0.68182	 0.71429	
0.82609		 0.60000	 0.81818	 0.69231	
0.69565		 0.45000	 0.75000	 0.56250	
0.63043		 0.11111	 0.66667	 0.19048	
0.71739		 0.70588	 0.60000	 0.64865	
0.56522		 0.13333	 0.22222	 0.16667	
0.56522		 0.52941	 0.42857	 0.47368	
0.67391		 0.18182	 0.25000	 0.21053	
0.71739		 0.35714	 0.55556	 0.43478	
0.71739		 0.23077	 0.50000	 0.31579	

Mean Accuracy:	 0.6848
Mean Precision:	 0.4049
Mean Recall:	 0.5473
Mean F_1:	 0.4410
